# Start - Datasets

In [1]:
%%time

# Step1 - Get repo
! rm -rf CS4180-DL
! git clone https://github.com/prerakmody/CS4180-DL

# Step2 - Get Dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtrainval_11-May-2012.tar --directory /content/CS4180-DL/data/dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtrainval_06-Nov-2007.tar --directory /content/CS4180-DL/data/dataset
! wget -P /content/CS4180-DL/data/dataset https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
! tar xf /content/CS4180-DL/data/dataset/VOCtest_06-Nov-2007.tar --directory /content/CS4180-DL/data/dataset
! mkdir /content/CS4180-DL/data/weights

! wget -P /content/CS4180-DL/data/weights https://pjreddie.com/media/files/yolov2-voc.weights

Cloning into 'CS4180-DL'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 21861 (delta 192), reused 185 (delta 91), pack-reused 21569
Receiving objects: 100% (21861/21861), 839.58 MiB | 39.43 MiB/s, done.
Resolving deltas: 100% (10252/10252), done.
--2019-06-14 14:44:10--  https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/octet-stream]
Saving to: ‘/content/CS4180-DL/data/dataset/VOCtrainval_11-May-2012.tar’

VOCtrainval_11-May- 100%[===================>]   1.86G  4.49MB/s    in 24m 24s 

2019-06-14 15:08:35 (1.30 MB/s) - ‘/content/CS4180-DL/data/dataset/VOCtrainval_11-May-2012.tar’ saved [1999639040/1999639040]

--2019-06-14 15:09:09--  https://pjreddie.com/

In [2]:
import os
import sys

%load_ext autoreload
%autoreload 2

## Step3 - Add path to main dir
DIR_MAIN = os.path.abspath('CS4180-DL/') #'../../../CS4180-DL/'
sys.path.append(DIR_MAIN)

## Step4 - Download pruned weights from GDrive
from src.utils import download_gdrive
# FILE_ID =  '1dHnUQ8G3GObZSMh9eQ0zUR5wor0ttW3U' # weight-pruning
# FILE_ID = '1I9FE3X5luVTQ0p9j8g1RWy63t5NiRWw_' # filter-pruning
# DIR_WEIGHTS = os.path.join(DIR_MAIN, 'data/weights/pruned')
# ! mkdir {DIR_WEIGHTS}
# DEST_NAME_ZIP   = os.path.join(DIR_WEIGHTS, 'weights-prune-me.zip')
# DEST_NAME_FILES = DIR_WEIGHTS
# download_gdrive(FILE_ID, DEST_NAME_ZIP)
# ! unzip {DEST_NAME_ZIP} -d {DEST_NAME_FILES}

## Step5 - Generate .txt files for training/validation 
from src.dataloader import setup_VOC
DIR_DATA = os.path.join(DIR_MAIN, 'data/dataset/')
setup_VOC(DIR_DATA)
DIR_DATA_VOC = os.path.join(DIR_DATA, 'VOCdevkit')
! cat {DIR_DATA_VOC}/2007_train.txt {DIR_DATA_VOC}/2007_val.txt {DIR_DATA_VOC}/2012_*.txt > {DIR_DATA_VOC}/voc_train.txt

## Step6 - Check which GPU
! nvidia-smi

 - year :  2012  || image_set :  train
 - year :  2012  || image_set :  val
 - year :  2007  || image_set :  train
 - year :  2007  || image_set :  val
 - year :  2007  || image_set :  test
Fri Jun 14 15:24:17 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                          

# Start - Code

In [3]:
import os
import sys

import torch

dir_main = os.path.abspath('CS4180-DL')
sys.path.append(dir_main)
print (' - In Path : ', sys.path[-1])

USE_GPU = torch.cuda.is_available()
print (' - USE_GPU : ', USE_GPU)

!nvidia-smi

 - In Path :  /content/CS4180-DL
 - USE_GPU :  True
Fri Jun 14 15:24:32 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    32W / 149W |     11MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                        

# Start - Predict

In [4]:
torch.cuda.empty_cache()
from src.predict import *
from src.nets import *

from src.pruning.weightPruning.methods import weight_prune,quick_filter_prune
from src.pruning.weightPruning.utils import prune_rate

from tensorboardcolab import TensorBoardColab

if (1):
    DIR_PROJ         = 'CS4180-DL'

if (1):
    MODEL            = ''
    MODEL_CFGFILE    = os.path.join(DIR_PROJ, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
    
    MODEL_WEIGHTFILE = os.path.join(DIR_PROJ, 'data/weights/yolov2-voc.weights') #['0.5' : 0.6366, '0.25' : 0.7, '0.1' : 0.7363]
    EVAL_PREFIX      = 'iter1_pretrained_'
    
    MODEL_LOSS       = RegionLoss()
    
    PASCAL_DIR       = os.path.join(DIR_PROJ, 'data/dataset/VOCdevkit/')
    EVAL_IMAGELIST   = os.path.join(DIR_PROJ, 'data/dataset/VOCdevkit/2007_test.txt')
    EVAL_OUTPUTDIR   = os.path.join(DIR_PROJ, 'eval_data')
    EVAL_OUTPUTDIR_PKL = os.path.join(DIR_PROJ, 'eval_results')

    
    print (' - 0. MODEL       : ', MODEL)
    print (' - 0. MODEL_WEIGHT: ', MODEL_WEIGHTFILE)
    print (' - 0. EVAL_PREFIX : ', EVAL_PREFIX)
    
if (1):
    BATCH_SIZE = 32
    CONF_THRESH = 0.05 # [0.5, 0.25, 0.1, 0.005]
    print (' - 0. BATCH_SIZE  : ', BATCH_SIZE)
    print (' - 0. CONF_THRESH : ', CONF_THRESH)
    
if (1):
    try:
        print (' - 0. Logger      : ', LOGGER)
    except:
        LOGGER = TensorBoardColab()
        print (' - 0. Logger      : ', LOGGER)    
    
    print ('')


valObj = PASCALVOCEval(MODEL, MODEL_CFGFILE, MODEL_WEIGHTFILE, MODEL_LOSS 
                       ,PASCAL_DIR, EVAL_IMAGELIST, EVAL_OUTPUTDIR, EVAL_PREFIX, EVAL_OUTPUTDIR_PKL
                       , LOGGER)
valObj.predict(BATCH_SIZE=BATCH_SIZE, CONF_THRESH=CONF_THRESH)
# valObj._do_python_eval()

Using TensorFlow backend.


 - 0. MODEL       :  
 - 0. MODEL_WEIGHT:  CS4180-DL/data/weights/yolov2-voc.weights
 - 0. EVAL_PREFIX :  iter1_pretrained_
 - 0. BATCH_SIZE  :  32
 - 0. CONF_THRESH :  0.05
Wait for 8 seconds...
TensorBoard link:
https://441f3086.ngrok.io
 - 0. Logger      :  <tensorboardcolab.core.TensorBoardColab object at 0x7fa981349a20>

 - 1. Loading model :  CS4180-DL/data/weights/yolov2-voc.weights
  -- [DEBUG] Non-BN Block :  {'type': 'convolutional', 'batch_normalize': 0, 'size': '1', 'stride': '1', 'pad': '1', 'filters': '125', 'activation': 'linear'}
  -- [DEBUG][Darknet] self.anchors :  [1.3221, 1.73145, 3.19275, 4.00944, 5.05587, 8.09892, 9.47112, 4.84053, 11.2364, 10.0071]
  -- [DEBUG][Darknet] self.num_anchors :  5
  -- [DEBUG][Darknet] self.anchor_step :  2.0
  -- [DEBUG][Darknet] self.num_classes :  20
  -- [DEBUG][Darknet] self.loss :  RegionLoss()


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
W0614 15:29:47.181011 140368128042880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorboardcolab/core.py:49: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0614 15:29:47.193191 140368128042880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorboardcolab/core.py:101: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.




  -- Reading predictions from :  CS4180-DL/eval_data / iter1_pretrained_ *
Mean AP = 0.7429


# Start - Training

In [0]:
%reset
import os
import sys

import torch
from tensorboardcolab import TensorBoardColab

torch.cuda.empty_cache()

if (torch.cuda.is_available()):
    
    if (1):
        DIR_MAIN         = os.path.abspath('CS4180-DL')
        sys.path.append(DIR_MAIN)
        print (' - 0. DIR_MAIN     :  ', DIR_MAIN)
        from src.train import YOLOv2Train
        
    if (1):
        PASCAL_DIR   = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/')
        PASCAL_TRAIN = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/voc_train.txt')
        PASCAL_VALID = os.path.join(DIR_MAIN, 'data/dataset/VOCdevkit/2007_test.txt')
        TRAIN_LOGDIR = os.path.join(DIR_MAIN, 'train_data')
        VAL_LOGDIR   = os.path.join(DIR_MAIN, 'eval_data')
        VAL_OUTPUTDIR_PKL = os.path.join(DIR_MAIN, 'eval_results')
        MODEL_CFG    = os.path.join(DIR_MAIN, 'data/cfg/github_pjreddie/yolov2-voc.cfg')
        
    if (1):
        
        MODEL_WEIGHT = os.path.join(DIR_MAIN, 'data/weights/yolov2-voc.weights')
        VAL_PREFIX   = 'pretrained_'
        
        print (' - 0. MODEL_WEIGHT :  ', MODEL_WEIGHT)
        print (' - 0. VAL_PREFIX   : ', VAL_PREFIX)
        
    if (1):
        BATCH_SIZE    = 32;
        print (' - 0. BATCH_SIZE    : ', BATCH_SIZE)
        
    if (1):
        DEBUG_EPOCHS = -1 #[-1, 50]
        MAX_EPOCHS   = 100
        
        print (' - 0. DEBUG_EPOCHS : ', DEBUG_EPOCHS)
        print (' - 0. MAX_EPOCHS   : ', MAX_EPOCHS)

    if (1):
        LEARNING_RATE = 0.00025
        print (' - 0. LEARNING_RATE : ', LEARNING_RATE)
        LEARNING_RATES = [LEARNING_RATE for _ in range(MAX_EPOCHS)]
    else:
        import numpy as np
        a = np.linspace(0.0005,0.0001, 10)
        b = np.linspace(0.0001, 0.00001, 10)
        c = np.linspace(0.00001, 0.0001, 10)
        d = np.linspace(0.0001, 0.0005,10) 
        LEARNING_RATES = np.hstack((a,b,c,d,a,b,c,d,a,b))

        import matplotlib.pyplot as plt
        plt.plot(LEARNING_RATES)
        plt.title('Cyclical Learning Rates')
        plt.xlabel('Epochs')
        plt.ylabel('Learning Rate')
        plt.show()
    
    assert len(LEARNING_RATES) == MAX_EPOCHS
    
    if (1):
        pruning_perc   = 20. #[0, 10., 30., 50., 70., 90.]
        pruning_method = "filter" # ["filter", "weight"]
    
    if (1):
        try:
            LOGGER = ''
            print (' - 0. Logger       : ', LOGGER)
        except:
            LOGGER = TensorBoardColab()
            print (' - 0. Logger       : ', LOGGER)
        print ('')
    

    if (1):
        trainObj = YOLOv2Train()
        trainObj.train(PASCAL_DIR, PASCAL_TRAIN, PASCAL_VALID, TRAIN_LOGDIR, VAL_LOGDIR, VAL_OUTPUTDIR_PKL, VAL_PREFIX
                       , MODEL_CFG, MODEL_WEIGHT
                       , BATCH_SIZE, LEARNING_RATES, MAX_EPOCHS
                       , LOGGER, DEBUG_EPOCHS=DEBUG_EPOCHS, verbose=0
                       , pruning_perc=pruning_perc, pruning_method=pruning_method
                      )
else:
    print (' - GPU Issues!!')